In [5]:
import pandas as pd
import pyodbc

cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};Server=lasr-sqldb-prd-mss,17001;Database=MSS_S_3PD_NS;Trusted_Connection=yes')

Error: ('08001', '[08001] [Microsoft][SQL Server Native Client 11.0]TCP Provider: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n (10060) (SQLDriverConnect)')

In [3]:
df2=pd.read_sql("""
select 
a11.*,
t1.terr_nm as WHSLR_TERR_NM,
t2.terr_nm as RPC_TERR_NM,
t3.terr_nm as IRM_TERR_NM,
b.SHARE_CLASS_NM,
c.PLTFM_MODL_PORT_PRGRM,
E.FRST_NM,
E.LAST_NM
from MSS_I_MSSBIBRTXN_S.DBO.FINANCIAL_TRANSACTION  a11
--join MSS_I_MSSBI_NS.DBO.FINCL_ITRMY_ORG_BROKER_VW a13 on (a11.CRDTD_TO_ORG_ID = a13.FINCL_ITRMY_ORG_ID)
join MSS_I_MSSBI_NS.DBO.FUND_SHARE_CLASS_VW b on (a11.FUND_SHARE_CLASS_ID = b.FUND_SHARE_CLASS_ID)
join MSS_I_MSSBI_NS.DBO.LU_PLTFM_MODL_PORT_VW C on (C.PLTFM_MODL_PORT_CD = a11.PLTFM_MODL_PORT_CD)
join MSS_I_MSSBI_NS.DBO.FINCL_ITRMY_PERSON_VW E on (E.FINCL_ITRMY_PRSON_UID = a11.FINCL_ITRMY_PRSON_UID)
join MSS_I_MSSBIBRDIM_NS.DBO.INVESTMENT_VEHICLE d on a11.INV_VEH_ID = d.INV_VEH_ID
LEFT join (
            SELECT DISTINCT TERR_ID,SLS_TERR_COVRG_TYP_NM,TERR_NM
            FROM
            (
            SELECT *,DENSE_RANK() OVER (PARTITION BY TERR_ID ORDER BY CR_DT DESC) DENSE_RANK_NUM
            FROM [MSS_I_MSSBI_NS].[DBO].TERRITORY_DIM_VW
            ) A WHERE DENSE_RANK_NUM = 1
          ) as t1 ON t1.TERR_ID = a11.WHSLR_TERR_ID
LEFT join (
            SELECT DISTINCT TERR_ID,SLS_TERR_COVRG_TYP_NM,TERR_NM
            FROM
            (
            SELECT *,DENSE_RANK() OVER (PARTITION BY TERR_ID ORDER BY CR_DT DESC) DENSE_RANK_NUM
            FROM [MSS_I_MSSBI_NS].[DBO].TERRITORY_DIM_VW
            ) A WHERE DENSE_RANK_NUM = 1
          ) as t2 ON t2.TERR_ID = a11.RPC_TERR_ID
LEFT join (
            SELECT DISTINCT TERR_ID,SLS_TERR_COVRG_TYP_NM,TERR_NM
            FROM
            (
            SELECT *,DENSE_RANK() OVER (PARTITION BY TERR_ID ORDER BY CR_DT DESC) DENSE_RANK_NUM
            FROM [MSS_I_MSSBI_NS].[DBO].TERRITORY_DIM_VW
            ) A WHERE DENSE_RANK_NUM = 1
          ) as t3 ON t3.TERR_ID = a11.IRM_TERR_ID
where a11.MO_NUM >= 201805
AND a11.TXN_UID in
(
4601427699,
4601340035,
4601421191,
4601334799,
4601425766,
4601341240,
4601426773,
4601339674,
4601428949,
4601347390,
4601432800,
4601352966,
4601424385,
4601327443,
4601423498,
4601327007,
4601404527,
4601345082,
4601431489,
4601349103,
4601379394,
4601426763,
4601383267,
4601428938,
4601378444,
4601427689,
4601375342,
4601425756,
4601376432,
4601421182,
4601385643,
4601432791,
4601372596,
4601423489,
4601384621,
4601374946,
4601424380,
4601431480,
4601374687,
4601432997
)
--AND Convert(varchar,a11.STTLD_DT,112) IN  (20180508)
--AND a11.TXN_TYP_CD IN (1101, 1102,1103,1104)
AND a11.RPT_SET_CD IN (101,103,105,107,109,113)
--AND a11.CRDTD_TO_ORG_ID in (106,306,1105,64)
--AND ABS(a11.GROSS_AMT) > 100000
ORDER BY ABS(a11.GROSS_AMT) DESC
""",cnxn)


In [6]:
from openpyxl import load_workbook

book = load_workbook('C:/Users/KPMYAD/Desktop/SAMPLING/Txn_Detail_Template_20180614Python_NADDG620.xlsx')
writer = pd.ExcelWriter('C:/Users/KPMYAD/Desktop/SAMPLING/Txn_Detail_Template_20180614Python_NADDG620.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, "data")
# Close the Pandas Excel writer and output the Excel file.
writer.save()